# Dataframe

In [20]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/ghazafm/SocialMediaSentiment/main/preprocessing/Training/data/clean/pos_tagging/gabungan.csv')
df = df.dropna()

# splitting & under sampling

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24333 entries, 0 to 24332
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet         24333 non-null  object
 1   no_double     24333 non-null  object
 2   no_alay       24333 non-null  object
 3   pos           24333 non-null  object
 4   clean_manual  24333 non-null  object
 5   stemmed       24333 non-null  object
 6   no_stopwords  24333 non-null  object
 7   tag           24333 non-null  object
 8   label         24333 non-null  object
 9   cc            24333 non-null  int64 
 10  cd            24333 non-null  int64 
 11  dt            24333 non-null  int64 
 12  fw            24333 non-null  int64 
 13  in            24333 non-null  int64 
 14  jj            24333 non-null  int64 
 15  md            24333 non-null  int64 
 16  neg           24333 non-null  int64 
 17  nn            24333 non-null  int64 
 18  nnd           24333 non-null  int64 
 19  nnp 

In [114]:
def undersampling(x):
    x_p = x[x['label']=='Positive']
    x_n = x[x['label']=='Negative']

    x_temp = x_p.sample(x_n.label.count(),random_state=42)
    x_under = pd.concat([x_temp,x_n],axis=0,ignore_index=True)
    return x_under

In [115]:
data = undersampling(df)

In [54]:
def tfidf_vec(data):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    data = vectorizer.fit_transform(data)
    data = data.toarray()
    data = pd.DataFrame(data, columns=vectorizer.get_feature_names_out())
    return data

In [55]:
tag = df[df.columns[-20:]]
x = df['no_stopwords']
x = tfidf_vec(x)
y = df['label']

# y.value_counts()

In [63]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24333 entries, 0 to 24332
Columns: 18125 entries, ab to zumba
dtypes: float64(18125)
memory usage: 3.3 GB


In [19]:
x,y = undersampling(x,y)

# vectorizing

In [23]:
# x_train, x_test = tfidf_vec(x_train, x_test)

# Modeling

In [24]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(random_state=42)



parameter = {
    "criterion": ["gini", "entropy"],
    "max_depth": range(1,10),
    "min_samples_split": range(2,10),
    "min_samples_leaf": range(1,5)
}

# from sklearn.metrics import recall_score, make_scorer

# # Membuat scorer untuk recall positif
# scorer = make_scorer(recall_score, pos_label='Positive')

from sklearn.metrics import precision_score, make_scorer

# Membuat scorer untuk precision
scorer = make_scorer(precision_score, pos_label='Positive')


from sklearn.model_selection import GridSearchCV
GS = GridSearchCV(estimator = tree,
                    param_grid = parameter,
                    scoring = scorer, 
                    refit = False,
                    cv=5,
                    verbose = 3)

GS.fit(x_train, y_train)
print(GS.best_params_)


Fitting 5 folds for each of 576 candidates, totalling 2880 fits
[CV 1/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=2;, score=0.641 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=2;, score=0.643 total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=2;, score=0.643 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=2;, score=0.642 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=2;, score=0.642 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=3;, score=0.641 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=3;, score=0.643 total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=3;, score=0.643 total time=   0.5s
[CV 4/5] END cri

In [21]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=42, criterion="gini", max_depth=9, min_samples_leaf=2, min_samples_split=7)
tree.fit(x_train, y_train)
    
y_pred = tree.predict(x_test)

# Score Decision Tree with tag

In [22]:
print(all_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.72      0.08      0.14      1718
    Positive       0.66      0.98      0.79      3147

    accuracy                           0.66      4865
   macro avg       0.69      0.53      0.47      4865
weighted avg       0.68      0.66      0.56      4865



In [ ]:
import pickle

# save the model to disk
filename = 'DecisionTree_Tag.pkl'
pickle.dump(tree, open(filename, 'wb'))

In [ ]:
from sklearn.metrics import f1_score

# Menghitung F1-score dengan label positif "1"
f1 = f1_score(y_test, y_pred, pos_label='Negative')

# Mencetak hasil
print("F1 Score:", f1)


F1 Score: 0.6698564593301436
